# Exercise 01 - CDS Feature Comparisons <img src="images/JHI_STRAP_Web.png" style="width: 150px; float: right;">

## Introduction

We often have particular genes/proteins of interest, and want to identify these - or equivalent sequences - in our newly-sequenced genomes. We may also want to identify corresponding sequences in bulk across our newly-sequenced genomes. In this notebook, we will look at three methods (among many others!) of identifying equivalent sequence features in genomes, in bulk.

The methods we will look at can broadly be considered to be of three different types:

* one-way pairwise comparison
* two-way pairwise comparison
* clustering

and they will give different results, for the same data.

## Running cells in this notebook

This is an interactive notebook, which means you are able to run the code that is written in each of the cells.

To run the code in a cell, you should:

1. Place your mouse cursor in the cell, and click (this gives the cell *focus*) to make it active
2. Hold down the `Shift` key, and press the `Return` key.

If this is successful, you should see the input marker to the left of the cell change from

```
In [ ]:
```

to (for example)

```
In [1]:
```

and you may see output appear below the cell.

### Requirements

<div class="alert alert-success">
To complete this exercise, you will need:
<ul>
<li>an active internet connection
<li>a local installation of [`BLAST+`](https://blast.ncbi.nlm.nih.gov/Blast.cgi?PAGE_TYPE=BlastDocs&DOC_TYPE=Download)
<li>a local installation of [`OrthoFinder`])(https://github.com/davidemms/OrthoFinder/releases)
</ul>
</div>

## One-Way Best `BLAST` matches (BBH)

It is still common to see one-way matches used - even if only informally, or as a first attempt - as a means of identifying equivalent proteins/features in a genome. In this section, we'll carry out a one-way `BLAST` search between the protein complements of *P. syringae* B728a and *P. fluorescens* NCIMB 11764, and inspect the results graphically.

### Performing the `BLASTP` query

We will use the `blastp` command at the terminal to use every protein sequence in the *P. syringae* B728a annotation as a query against the predicted proteome of *P. fluorescens* NCIMB 11764. We will use some custom settings to make our analysis easier to carry out.

<div class="alert alert-warning">
<ul>
<li> We will want to limit our matches to only the best hit, so we specify `-max_target_seqs 1` 
<li> We want our output in tab-separated tabular particular format so we can import it easily, and we want some specific non-standard columns (e.g. query sequence coverage) in that table so we can carry out some useful calculations and visualisation. We therefore specify `-outfmt "6 qseqid sseqid qlen slen length nident pident qcovs evalue bitscore"`
<li> To make the comparisons quicker, we should create `BLAST` databases for each of the three proteomes, with the `makeblastdb` command. 
</ul>
</div>

The relevant `BLAST` databases have already been created for you to save time (using the `scripts/02-cds_feature_comparisons.sh` script), and the results are in the `pseudomonas_blastp` directory:

```
$ tree ./pseudomonas_blastp
./pseudomonas_blastp
├── GCF_000012245.1_ASM1224v1_protein.phr
├── GCF_000012245.1_ASM1224v1_protein.pin
├── GCF_000012245.1_ASM1224v1_protein.psq
├── GCF_000293885.2_ASM29388v3_protein.phr
├── GCF_000293885.2_ASM29388v3_protein.pin
├── GCF_000293885.2_ASM29388v3_protein.psq
├── GCF_000988485.1_ASM98848v1_protein.phr
├── GCF_000988485.1_ASM98848v1_protein.pin
└── GCF_000988485.1_ASM98848v1_protein.psq
```

To carry out the one-way `BLASTP` search of *P. syringae* B728a against *P. fluorescens* NCIMB 11764, execute the following command in the terminal:

```
blastp -query  pseudomonas/GCF_000988485.1_ASM98848v1_protein.faa \
       -db pseudomonas_blastp/GCF_000293885.2_ASM29388v3_protein \
       -max_target_seqs 1 \
       -outfmt "6 qseqid sseqid qlen slen length nident pident qcovs evalue bitscore" \
       -out pseudomonas_blastp/B728a_vs_NCIMB_11764.tab
```

This will take a couple of minutes to complete.

### Importing and visualising the results

The `Python` module `helpers` is included in this directory, to provide useful helper functions so that we can read and view the `BLASTP` output generated above. To make the functions available, we import it by running the `Python` code cell below.

<div class="alert alert-warning">
<b>NOTE:</b> The `%pylab inline` "magic" below allows us to see plots of the `BLAST` data we load.
</div>

In [ ]:
%pylab inline
# Import helper module
from helpers import ex02

The first thing we do is load in the `BLASTP` output we generated, so that we can plot some of the key features. We do that using the `ex02.read_data()` function in the cell below:

In [ ]:
# Load one-way BLAST results into a data frame called data_fwd
data_fwd = ex02.read_data("pseudomonas_blastp/B728a_vs_NCIMB_11764.tab")

<div class="alert alert-warning">
<b>NOTE:</b> In the cell below, the `data.head()` function shows us the first few lines of the one-way `BLASTP` results, one per match; the `data.describe()` function shows us some summary data for the table.
</div>

In [ ]:
# Show first few lines of the loaded data
data_fwd.head()

In [ ]:
# Show descriptive statistics for the table
data_fwd.describe()

There are 5265 rows in this table, one for each of the query protein sequences in the *P. syringae* B728a annotation.

We can look at the distribution of values in the dataframe rows using the `.hist()` method for any column of interest. For example, `data_fwd.subject_length.hist()` plots a histogram of the values in the `subject_length` column.

<div class="alert alert-warning">
<b>NOTE:</b> The `bins=100` option sets the number of value bins used in the histogram
</div>

In [ ]:
# Plot a histogram of alignment lengths for the BLAST data
data_fwd.alignment_length.hist(bins=100)

In [ ]:
# Plot a histogram of percentage identity for the BLAST data
data_fwd.identity.hist(bins=100)

In [ ]:
# Plot a histogram of query_coverage for the BLAST data
data_fwd.query_coverage.hist(bins=100)

In [ ]:
# Plot a histogram of percentage coverage for the BLAST data
data_fwd.subject_coverage.hist(bins=100)

From these histograms, we can see that:

* most alignments are shorter than 500 amino acids long
* that the alignment covers more than 80% of nearly all queries
* that the alignment covers more than 80% of nearly all best matches to each query
* there is a bimodal distribution of sequence identity: one where best matches peak at ≈80% identity, and one where best matches peak at ≈30% identity

<div class="alert alert-warning">
<ul>
<li><b>What size are most one-way best `BLAST` alignments?</b>
<li><b>What is the typical query coverage?</b>
<li><b>What is the typical subject coverage?</b>
<li><b>What is the typical best `BLAST` match identity?</b>
<li><b>Do one-way best `BLAST` matches always identify equivalent proteins?</b>
</ul>
</div>

We can view the relationship betweens query coverage and subject coverage, and query coverage and match identity for these one-way best `BLAST` hits by plotting a 2D histogram, with the helper function `ex02.plot_hist2d()` in the cell below.

In [ ]:
# Plot 2D histogram of subject sequence (match) coverage against query
# sequence coverag
ex02.plot_hist2d(data_fwd.query_coverage, data_fwd.subject_coverage,
                "one-way query COV", "one-way subject COV", 
                "one-way coverage comparison")
ex02.plot_hist2d(data_fwd.query_coverage, data_fwd.identity,
                "one-way query COV", "one-way match PID", 
                "one-way coverage/identity comparison")

<div class="alert alert-warning">
<ul>
<li>**What is the query/subject coverage for most one-way best `BLAST` matches?**
<li>**Do the remaining one-way `BLAST` matches have the same coverage for query and subject?**
<li>**What is the typical query coverage of a high percentage identity match?**
<li>**What is the typical query coverage of a low percentage identity match?**
</ul>
</div>